# Initialization

Test notebook for the C-MAPPS benchmark. Get best parameters for each dataset. 

First we import the necessary packages and create the global variables.

In [2]:
import math
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

# Define the model

In [3]:
def RULmodel_SN(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Define objective function

Define the function that evaluates each set of data-related params and returns the RMSE as value.

In [6]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel("RUL_SN_optmizable", modelRULSN, selected_features, dataFolder = '../CMAPSSData',
                          scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData(rectify_labels = False)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.trainTime
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateModel(["rhs"], round=2)
    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

# Optimize the parameters for the NN using DE

In [ ]:
#Optimize the parameters for the NN using DE

maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
totalTime = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

#datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("intermediateResults3_notRectified"+datasetNumber+".txt", "w")

    windowSizeBounds = [1,maxWindowSize[datasetNumber]]
    windowStrideBounds = [1,10]
    constantRULBounds = [90,140]

    bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]


    startTime = time.clock()
    tempResults = differential_evolution(nn_optmize_fun, bounds, 
                                     args=(selected_features, datasetNumber, min_max_scaler, 0, 20, file, 1),
                                    strategy='best1bin', maxiter=10, popsize=20, disp=True)
    results[datasetNumber] = tempResults
    endTime = time.clock()

    file.close()
    totalTime[datasetNumber] = endTime - startTime

Tunning for dataset 2
Instructions for updating:
keep_dims is deprecated, use keepdims instead
259/259 [==============================] - 0s 112us/step


In [8]:
print("Total time {}".format(totalTime))
print(results)

Total time {'2': 5482.939638694275, '1': 2813.138622107447, '4': 7551.063892150514, '3': 3423.4927144935773}
{'2':      fun: 32.317815845499496
 message: 'Optimization terminated successfully.'
    nfev: 632
     nit: 8
 success: True
       x: array([ 17.21957661,   1.50376803, 135.0177935 ]), '1':      fun: 15.171459105396513
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 744
     nit: 10
 success: False
       x: array([ 25.62259455,   1.26541061, 135.34659429]), '4':      fun: 37.65520767405298
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 772
     nit: 10
 success: False
       x: array([ 12.90196629,   1.43064081, 137.69058978]), '3':      fun: 16.869726896037026
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 776
     nit: 10
 success: False
       x: array([ 29.85369584,   1.87892566, 131.98160269])}
